In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import scvelo as scv
from scipy import stats
import os
import matplotlib
import pickle
%matplotlib inline
import palantir
import harmony

import random
np.random.seed(0)
random.seed(0)

# Human data

In [263]:
human_counts = pd.read_csv('filtered_normalized_counts_human_SCP.txt', '\t', index_col = 0)
human_ann = pd.read_csv('xy_human_v2.txt', '\t', index_col = 0)
human_ann.drop('TYPE', inplace = True)
human_counts = human_counts.T
human_counts = harmony.utils.normalize_counts(human_counts)
human_counts['Cluster'] = human_ann.loc[human_counts.index].Cluster
human_mean = human_counts.replace(0, np.NaN).groupby('Cluster').mean().replace(np.NaN,0)
human_mean

GENE,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
Cluster,,,,,,,,,,,,,,,,,,,,,
Basal,1.098823,1.168770,1.617591,1.553406,0.565547,1.400608,0.878194,0.000000,2.834659,0.943599,...,1.422939,1.442678,1.243282,1.862716,1.543710,1.084502,1.567206,2.562406,1.359520,1.554398
Brush+PNEC,1.805380,0.000000,1.065417,0.000000,0.000000,1.142464,0.000000,0.000000,3.014566,0.000000,...,2.071030,1.661298,1.670475,1.192979,1.148924,1.115146,1.347076,1.570428,2.268203,1.720718
Ciliated,1.756412,1.367611,1.221450,0.000000,1.107052,1.618596,1.530222,2.521273,2.081935,1.195886,...,1.242340,1.282893,1.654666,1.294820,1.514541,1.743931,1.545904,1.773143,1.569853,1.469112
FOXN4+,0.000000,0.000000,2.621919,0.000000,0.000000,1.612791,0.727061,0.000000,2.082268,0.000000,...,1.241051,1.213214,2.237427,0.824329,1.251608,0.000000,3.306545,2.253164,0.803277,1.406366
Interm. basal>secr.,1.129929,0.987943,1.372880,1.213262,1.356899,1.424918,0.738030,1.097939,2.904282,1.039824,...,1.385008,1.501951,1.338031,1.491313,1.567940,1.284654,1.786060,2.198739,1.328837,1.630534
Interm. secr.>cil.,0.786565,2.976188,1.282745,2.251745,1.305313,1.592425,1.372771,0.000000,2.582716,1.606168,...,1.816662,1.556216,1.614451,1.187230,1.921307,1.598091,1.903112,1.945184,1.380990,1.723451
Ionocytes,0.000000,0.000000,2.302708,0.000000,0.000000,0.000000,0.000000,2.394063,2.311600,0.000000,...,0.000000,0.000000,3.264773,2.067764,1.835813,0.000000,2.110491,3.421699,2.652984,2.625476
SLC16A7+,0.000000,0.000000,2.024214,2.342469,3.576538,3.158053,0.000000,0.000000,3.347809,0.000000,...,4.868549,0.000000,0.000000,2.704164,2.711186,4.814126,4.274712,1.971220,3.504924,0.000000
Secretory,0.000000,0.787717,1.299999,2.570033,0.698431,1.187542,0.000000,0.983435,1.982062,0.000000,...,1.271152,1.091288,0.901694,1.606969,1.143244,0.907868,1.419422,1.422108,1.213891,1.337081


# Mouse data

In [264]:
mouse_counts = pd.read_csv('filtered_normalized_counts_mouse_SCP.txt', '\t', index_col = 0)
mouse_ann = pd.read_csv('xy_mouse_all_v2.txt', '\t', index_col = 0)
mouse_ann.drop('TYPE', inplace = True)
mouse_counts = mouse_counts.T
mouse_counts = harmony.utils.normalize_counts(mouse_counts)
mouse_counts['Cluster'] = mouse_ann.loc[mouse_counts.index].Cluster
mouse_mean = mouse_counts.replace(0, np.NaN).groupby('Cluster').mean().replace(np.NaN,0)
#mouse_mean

In [265]:
#pd.Series(mouse_counts.columns.to_list()).to_csv('mouse_genes.tsv', sep = '\t', index = False)
h_m_mapper = pd.read_csv('../mouse_human_map.txt', sep = '\t', index_col = 1)['Gene name'].to_dict()
mouse_counts.columns = mouse_counts.columns.map(h_m_mapper)
mouse_counts = mouse_counts.loc[:,~mouse_counts.columns.isna()]

In [266]:
mouse_counts

GENE,KIAA1841,C17orf49,C2orf68,C4orf19,C4orf54,C11orf58,C9orf16,C9orf24,C11orf1,C3orf80,...,FAM189A1,FAM189A2,FAM189B,FAM193A,FAM193B,FAM199X,FAM204A,FAM207A,FAM209B,FAM20A
MOUSE_CELL_0,0.0,0.000000,0.0,0.000000,0.0,0.000000,3.360215,0.0,0.000000,0.0,...,0.0,0.0,0.000000,1.680108,1.680108,0.000000,0.000000,1.680108,0.0,1.680108
MOUSE_CELL_1,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
MOUSE_CELL_2,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
MOUSE_CELL_3,0.0,0.000000,0.0,0.000000,0.0,0.000000,14.482259,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
MOUSE_CELL_4,0.0,1.596169,0.0,0.000000,0.0,0.000000,1.596169,0.0,0.000000,0.0,...,0.0,0.0,1.596169,0.000000,1.596169,1.596169,3.192338,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MOUSE_CELL_14158,0.0,0.000000,0.0,9.460738,0.0,9.460738,28.382214,0.0,0.000000,0.0,...,0.0,0.0,0.000000,9.460738,0.000000,0.000000,0.000000,0.000000,0.0,NaN
MOUSE_CELL_14159,0.0,0.000000,0.0,3.617945,0.0,0.000000,0.000000,0.0,3.617945,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN
MOUSE_CELL_14160,0.0,5.753740,0.0,5.753740,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,5.753740,0.000000,0.000000,0.000000,0.000000,0.0,NaN
MOUSE_CELL_14161,0.0,0.000000,0.0,13.227513,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,NaN


In [267]:
mouse_counts = mouse_counts.T
mouse_counts = mouse_counts.reset_index().groupby('GENE').mean()
mouse_counts = mouse_counts.T

### Map to human

In [268]:
pd.Series(mouse_mean.columns.to_list()).to_csv('mouse_genes.tsv', sep = '\t', index = False)
h_m_mapper = pd.read_csv('../mouse_human_map.txt', sep = '\t', index_col = 1)['Gene name'].to_dict()
mouse_mean.columns = mouse_mean.columns.map(h_m_mapper)
mouse_mean = mouse_mean.loc[:,~mouse_mean.columns.isna()]

In [247]:
mouse_mean.shape

(10, 4750)

In [269]:
mouse_mean = mouse_mean.T
mouse_mean = mouse_mean.reset_index().groupby('GENE').mean()
mouse_mean = mouse_mean.T

In [270]:
mouse_mean.shape

(10, 4570)

# Xenopus

In [271]:

def save_intermediate_files(datasets, temp_dir, file_name_template, identifier):
    for d in datasets:
        file = os.path.join('.', temp_dir, file_name_template + '_' + d.obs[identifier][0] + '.h5ad')
        d.write(file, compression='gzip')
        
identifiers = ['st08','st10.5','st12','st13','st16','st18','st20','st22','st24','st27']
      
def load_intermediate_files(temp_dir, file_name_template, identifiers):
    ob_list = []
    for i in identifiers:
        file = os.path.join('.', temp_dir, file_name_template + '_' + i+ '.h5ad')
        ob_list.append(sc.read_h5ad(file))
    return ob_list

import sys
sys.path.append('/work/Xenopus/Notebooks/V3/Integrated_analysis/')
from Integrate import concat_for_harmony, adata_to_df

timepoints = ['st08', 'st10.5', 'st12', 'st13', 'st16', 'st18', 'st20', 'st22', 'st24', 'st27']
adatas = load_intermediate_files( '/work/Xenopus/Notebooks/V3/intermediate_files', 'zUMI_raw_singlets' , timepoints)
counts = concat_for_harmony(adatas, timepoints)
norm_df = harmony.utils.normalize_counts(counts)

def adata_hvg(adata, N_genes = 1000):
    print(adata.obs.stage[0])
    counts = pd.DataFrame(data = adata.X.todense(), 
                          index = adata.obs_names,
                          columns = adata.var_names)
    norm_df = harmony.utils.normalize_counts(counts)
    hvg_genes = harmony.utils.hvg_genes(norm_df, no_genes=N_genes)
    return hvg_genes.tolist()

def flatten(ls):
    return [x for y in ls for x in y]
hvg_genes_per_data = list(set(flatten([adata_hvg(x, N_genes = 500) for x in adatas])))

st08
st10.5
st12
st13
st16
st18
st20
st22
st24
st27
Concatenating data..
st08
st10.5
st12
st13
st16
st18
st20
st22
st24
st27


In [272]:
df = pd.read_csv('/work/Xenopus/Notebooks/V3/misc_files/XENLA_XenBase20190115_prot.txt', sep = '\t', index_col = 0)
mapper = {x:h for x,h in zip(df.index, df['Unnamed: 5'])}
df_map = pd.DataFrame({'Data_name':counts.columns,
              'Reduced_name': [x.split('|')[1] for x in counts.columns]
             })
df_map['Human_name'] = df_map.Reduced_name.map(mapper)
mapper = {x:h for x, h in zip(df_map['Data_name'], df_map['Human_name'])}
norm_df = norm_df.loc[:,hvg_genes_per_data]
norm_df.columns = [mapper[x] for x in norm_df.columns]
counts = norm_df.T
counts = counts[counts.index!= 'NoName']
counts = counts[~counts.index.isna()]
#counts = counts.drop_duplicates(keep = 'first')

counts = counts.T
counts = counts.reset_index().groupby('index').mean()
counts = counts.T

import pickle

with open('/work/Xenopus/Notebooks/V3/intermediate_files/clusters', "rb") as f:
    clusters = pickle.load( f)

tp = pd.Series(index=norm_df.index)
for t in timepoints:
    cells = norm_df.index[norm_df.index.str.contains(t)]
    tp[cells] = t
    
anno = pd.concat([clusters, tp], axis = 1)
anno.columns = ['Clusters', 'Stage']    

AC_counts = counts.copy()
AC_anno = anno.copy()    

AC_counts = AC_counts.T

AC_counts['Clusters'] = AC_anno['Clusters']
AC_mean = AC_counts.groupby('Clusters').mean()#[common_genes]
AC_mean = AC_mean.replace(0, np.NaN).T.reset_index().groupby('index').mean().replace(np.NaN,0)
AC_mean = AC_mean.T


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [273]:
common_genes = list(set(mouse_mean.columns) & set(human_mean.columns) & set(AC_mean.columns))

In [274]:
human_mean = np.log1p(human_mean)
mouse_mean = np.log1p(mouse_mean)
AC_mean = np.log1p(AC_mean)